Predict future vehicle speed from:

Camera features (simulated CNN embeddings)

LiDAR features (simulated point cloud embeddings)

In [1]:
!pip install nuscenes-devkit

INFO: pip is looking at multiple versions of opencv-python-headless to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.0/316.0 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 MB 40.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 85.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.9/115.9 kB 7.9 MB/s eta 0:00:00
  Attempting uninstall: Shapely
    Found existing installation: shapely 2.1.2
    Uninstalling shapely-2.1.2:
      Successfully uninstalled shapely-2.1.2
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.12.0.88
    Uninstalling opencv-python-headless-4.12.0.88:
      Successfully uninstalled opencv-python-headless-4.12.0.88
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the 

In [2]:
!wget https://www.nuscenes.org/data/v1.0-mini.tgz

--2026-01-04 14:49:46--  https://www.nuscenes.org/data/v1.0-mini.tgz
Resolving www.nuscenes.org (www.nuscenes.org)... 13.227.219.18, 13.227.219.6, 13.227.219.45, ...
Connecting to www.nuscenes.org (www.nuscenes.org)|13.227.219.18|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4167696325 (3.9G) [application/x-tar]
Saving to: ‘v1.0-mini.tgz’

v1.0-mini.tgz       100%[===================>]   3.88G  36.9MB/s    in 2m 16s  

2026-01-04 14:52:03 (29.2 MB/s) - ‘v1.0-mini.tgz’ saved [4167696325/4167696325]



In [3]:
!tar -xf v1.0-mini.tgz

In [4]:
from nuscenes.nuscenes import NuScenes
# Ensure you have added the nuScenes-mini dataset to your Kaggle inputs
nusc = NuScenes(version='v1.0-mini', dataroot='/kaggle/working', verbose=True)

Loading NuScenes tables for version v1.0-mini...
23 category,
8 attribute,
4 visibility,
911 instance,
12 sensor,
120 calibrated_sensor,
31206 ego_pose,
8 log,
10 scene,
404 sample,
31206 sample_data,
18538 sample_annotation,
4 map,
Done loading in 0.499 seconds.
Reverse indexing ...
Done reverse indexing in 0.1 seconds.


In [5]:
nusc.list_scenes()

scene-0061, Parked truck, construction, intersectio... [18-07-24 03:28:47]   19s, singapore-onenorth, #anns:4622
scene-0103, Many peds right, wait for turning car, ... [18-08-01 19:26:43]   19s, boston-seaport, #anns:2046
scene-0655, Parking lot, parked cars, jaywalker, be... [18-08-27 15:51:32]   20s, boston-seaport, #anns:2332
scene-0553, Wait at intersection, bicycle, large tr... [18-08-28 20:48:16]   20s, boston-seaport, #anns:1950
scene-0757, Arrive at busy intersection, bus, wait ... [18-08-30 19:25:08]   20s, boston-seaport, #anns:592
scene-0796, Scooter, peds on sidewalk, bus, cars, t... [18-10-02 02:52:24]   20s, singapore-queensto, #anns:708
scene-0916, Parking lot, bicycle rack, parked bicyc... [18-10-08 07:37:13]   20s, singapore-queensto, #anns:2387
scene-1077, Night, big street, bus stop, high speed... [18-11-21 11:39:27]   20s, singapore-hollandv, #anns:890
scene-1094, Night, after rain, many peds, PMD, ped ... [18-11-21 11:47:27]   19s, singapore-hollandv, #anns:1762
sc

In [6]:
my_scene = nusc.scene[0]
my_scene

{'token': 'cc8c0bf57f984915a77078b10eb33198',
 'log_token': '7e25a2c8ea1f41c5b0da1e69ecfa71a2',
 'nbr_samples': 39,
 'first_sample_token': 'ca9a282c9e77460f8360f564131a8af5',
 'last_sample_token': 'ed5fc18c31904f96a8f0dbb99ff069c0',
 'name': 'scene-0061',
 'description': 'Parked truck, construction, intersection, turn left, following a van'}

In [7]:
nusc.sensor

[{'token': '725903f5b62f56118f4094b46a4470d8',
  'channel': 'CAM_FRONT',
  'modality': 'camera'},
 {'token': 'ce89d4f3050b5892b33b3d328c5e82a3',
  'channel': 'CAM_BACK',
  'modality': 'camera'},
 {'token': 'a89643a5de885c6486df2232dc954da2',
  'channel': 'CAM_BACK_LEFT',
  'modality': 'camera'},
 {'token': 'ec4b5d41840a509984f7ec36419d4c09',
  'channel': 'CAM_FRONT_LEFT',
  'modality': 'camera'},
 {'token': '2f7ad058f1ac5557bf321c7543758f43',
  'channel': 'CAM_FRONT_RIGHT',
  'modality': 'camera'},
 {'token': 'ca7dba2ec9f95951bbe67246f7f2c3f7',
  'channel': 'CAM_BACK_RIGHT',
  'modality': 'camera'},
 {'token': 'dc8b396651c05aedbb9cdaae573bb567',
  'channel': 'LIDAR_TOP',
  'modality': 'lidar'},
 {'token': '47fcd48f71d75e0da5c8c1704a9bfe0a',
  'channel': 'RADAR_FRONT',
  'modality': 'radar'},
 {'token': '232a6c4dc628532e81de1c57120876e9',
  'channel': 'RADAR_FRONT_RIGHT',
  'modality': 'radar'},
 {'token': '1f69f87a4e175e5ba1d03e2e6d9bcd27',
  'channel': 'RADAR_FRONT_LEFT',
  'modality'

In [13]:
import numpy as np
# List all 6 camera channels available in nuScenes
CAMERA_NAMES = ['CAM_FRONT', 'CAM_FRONT_RIGHT', 'CAM_BACK_RIGHT', 
                'CAM_BACK', 'CAM_BACK_LEFT', 'CAM_FRONT_LEFT']

def extract_full_system_data(nusc):
    dataset_index = []
    for sample in nusc.sample:
        # Collect all 6 camera paths into a dictionary
        cam_paths = {name: nusc.get_sample_data_path(sample['data'][name]) for name in CAMERA_NAMES}
        lidar_path = nusc.get_sample_data_path(sample['data']['LIDAR_TOP'])
        
        # Calculate velocity using ego-pose (your existing logic)
        cam_token = sample['data']['CAM_FRONT']
        current_pose = nusc.get('ego_pose', nusc.get('sample_data', cam_token)['ego_pose_token'])
        prev_sample_token = sample['prev']
        if prev_sample_token == '': continue
        prev_sample = nusc.get('sample', prev_sample_token)
        prev_pose = nusc.get('ego_pose', nusc.get('sample_data', prev_sample['data']['CAM_FRONT'])['ego_pose_token'])
        
        time_diff = (current_pose['timestamp'] - prev_pose['timestamp']) / 1e6
        dist = np.linalg.norm(np.array(current_pose['translation']) - np.array(prev_pose['translation']))
        velocity = dist / time_diff if time_diff > 0 else 0
        
        dataset_index.append({
            'cameras': cam_paths,
            'lidar_path': lidar_path,
            'speed': velocity
        })
    return dataset_index

sync_data = extract_full_system_data(nusc)

print(f"Total synchronized samples found: {len(sync_data)}")



Total synchronized samples found: 394


**Convert Lidar raw points to birds eye view .**


Converts nuScenes LiDAR bin file to a 2-channel BEV map.
boundary: (min_x, max_x, min_y, max_y) in meters

In [14]:
import torch

def lidar_bev(bin_path, grid_size=(224, 224), boundary=(-50, 50, -50, 50)):
    # 1. Load and slice points correctly
    scan = np.fromfile(bin_path, dtype=np.float32).reshape(-1, 5)
    points = scan[:, :3] 

    # 2. Filter points within boundary
    mask = (points[:, 0] >= boundary[0]) & (points[:, 0] < boundary[1]) & \
           (points[:, 1] >= boundary[2]) & (points[:, 1] < boundary[3])
    points = points[mask]

    # 3. Map meters to pixel indices
    x_px = ((points[:, 0] - boundary[0]) / (boundary[1] - boundary[0]) * grid_size[0]).astype(np.int32)
    y_px = ((points[:, 1] - boundary[2]) / (boundary[3] - boundary[2]) * grid_size[1]).astype(np.int32)

    # 4. Create BEV channels (Fast Vectorized Way)
    bev_map = np.zeros((2, grid_size[0], grid_size[1]), dtype=np.float32)
    
    # Normalize Z (height)
    norm_z = np.clip(points[:, 2] + 2, 0, 5) / 5.0

    # Instead of a loop, we use 'at' to handle multiple points hitting the same pixel
    # Channel 0: Max Height
    np.maximum.at(bev_map[0], (x_px, y_px), norm_z)
    
    # Channel 1: Density (Point Count)
    np.add.at(bev_map[1], (x_px, y_px), 1)

    # Normalize Density
    if bev_map[1].max() > 0:
        bev_map[1] = np.log1p(bev_map[1]) / np.log1p(bev_map[1].max())

    return torch.from_numpy(bev_map)
        

    
    
    


    



In [15]:
from torch.utils.data  import Dataset , DataLoader 
from PIL import Image 
import torchvision.transforms as T


class NuScenesFusion(Dataset):
    def __init__(self, sync_data):

        self.data = sync_data
        self.cam_transform = T.Compose([
            T.Resize((224,224)),
            T.ToTensor(),
            T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        
        # 1. Load all 6 images
        image_list = []
        for name in CAMERA_NAMES:
            img = Image.open(item['cameras'][name]).convert('RGB')
            image_list.append(self.cam_transform(img))
        
        # Make sure this name is used in the return statement!
        full_camera_tensor = torch.stack(image_list)
        
        # 2. Load LiDAR
        bev_tensor = lidar_bev(item['lidar_path'])
        
        # 3. Target Speed
        speed = torch.tensor([item['speed']], dtype=torch.float32)
        
        # FIX: Ensure these match the variables defined above
        return full_camera_tensor, bev_tensor, speed

dataset = NuScenesFusion(sync_data)
train_loader = DataLoader(dataset , batch_size=8 , shuffle = True)

    

Fusion model

In [16]:
import torch.nn as nn
from torchvision import models
class SurroundVelocityNet(nn.Module):
    def __init__(self):
        super().__init__()
        # Shared Camera Backbone: processes each image independently
        resnet = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
        self.cam_backbone = nn.Sequential(*list(resnet.children())[:-1])

        # LiDAR Branch
        self.lidar_branch = nn.Sequential(
            nn.Conv2d(2, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.AdaptiveAvgPool2d((1, 1))     
        )

        # Regressor: 512 features per camera * 6 cameras + 32 lidar features
        self.regressor = nn.Sequential(
            nn.Linear((512 * 6) + 32, 256),
            nn.ReLU(),
            nn.Linear(256, 1)
        )

    def forward(self, cams, bev):
        # cams shape: (Batch, 6, 3, 224, 224)
        b, n, c, h, w = cams.shape
        
        # Flatten batch and cameras to process through backbone at once
        cams = cams.view(-1, c, h, w) 
        cam_feats = self.cam_backbone(cams).view(b, n, -1) # (B, 6, 512)
        
        # Concatenate features from all 6 cameras
        cam_feats_combined = cam_feats.view(b, -1) # (B, 3072)
        
        # Process LiDAR
        lidar_feat = self.lidar_branch(bev).view(b, -1)
        
        # Final Fusion
        combined = torch.cat((cam_feats_combined, lidar_feat), dim=1)
        return self.regressor(combined)



        

In [19]:
# Grab one batch to test shapes
test_loader = DataLoader(dataset, batch_size=2)
test_cams, test_bevs, test_speeds = next(iter(test_loader))

print(f"Camera Batch Shape: {test_cams.shape}") # Expected: [2, 6, 3, 224, 224]
print(f"LiDAR Batch Shape: {test_bevs.shape}")  # Expected: [2, 2, 224, 224]
print(f"Speed Batch Shape: {test_speeds.shape}") # Expected: [2, 1]

# Test the model forward pass
test_output = model(test_cams.to(device), test_bevs.to(device))
print(f"Model Output Shape: {test_output.shape}") # Expected: [2, 1]

Camera Batch Shape: torch.Size([2, 6, 3, 224, 224])
LiDAR Batch Shape: torch.Size([2, 2, 224, 224])
Speed Batch Shape: torch.Size([2, 1])


NameError: name 'model' is not defined

In [20]:
import torch.optim as optim
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SurroundVelocityNet().to(device)

criterion = nn.HuberLoss(delta=1.0)
optimizer = optim.Adam(model.parameters(), lr=1e-4)

def train_model(model , train_loader , epochs=10):

    model.train()
    for epoch in range(epochs):
        loop = tqdm(train_loader , total=len(train_loader))
        epoch_loss = 0


        for cams, bevs, speeds in loop: # 'cams' now contains the 6-camera stack
            cams = cams.to(device)      # Shape: (Batch, 6, 3, 224, 224)
            bevs = bevs.to(device)
            speeds = speeds.to(device)
            
            # Zero gradients
            optimizer.zero_grad()
            
            # Forward pass
            predictions = model(cams, bevs)
            loss = criterion(predictions, speeds)
            
            # Backward pass & Optimization
            loss.backward()
            optimizer.step()
            
            epoch_loss += loss.item()
            loop.set_description(f"Epoch [{epoch+1}/{epochs}]")
            loop.set_postfix(loss=loss.item())
            
        print(f"Epoch {epoch+1} Complete. Avg Loss: {epoch_loss/len(train_loader):.4f}")

train_model(model, train_loader)




Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 218MB/s]
Epoch [1/10]: 100%|██████████| 50/50 [00:43<00:00,  1.15it/s, loss=3.64]  


Epoch 1 Complete. Avg Loss: 1.6538


Epoch [2/10]: 100%|██████████| 50/50 [00:42<00:00,  1.17it/s, loss=0.149] 


Epoch 2 Complete. Avg Loss: 0.5645


Epoch [3/10]: 100%|██████████| 50/50 [00:42<00:00,  1.17it/s, loss=0.715] 


...

Epoch 8 Complete. Avg Loss: 0.1612


Epoch [9/10]: 100%|██████████| 50/50 [00:42<00:00,  1.17it/s, loss=0.0101]


Epoch 9 Complete. Avg Loss: 0.1419


Epoch [10/10]: 100%|██████████| 50/50 [00:42<00:00,  1.18it/s, loss=0.0541] 

Epoch 10 Complete. Avg Loss: 0.1268


In [ ]:
import matplotlib.pyplot as plt

# --- A. Save the Model Weights ---
torch.save(model.state_dict(), 'K_fusion_model.pth')
print("Model saved as 'K_fusion_model.pth'")

# --- B. Plot 1: Training Convergence (Loss Curve) ---
plt.figure(figsize=(10, 5))
plt.plot(history, color='#3498db', label='Huber Loss')
plt.title('Fusion Model Training Convergence', fontsize=16)
plt.xlabel('Iterations', fontsize=12)
plt.ylabel('Loss', fontsize=12)
plt.grid(True, alpha=0.3)
plt.legend()
plt.savefig('loss_curve.png') # Saves plot as image
plt.show()

# --- C. Plot 2: Prediction Dashboard (The "Fixed" Version) ---
def final_dashboard(model, dataset, device):
    model.eval()
    idx = np.random.randint(0, len(dataset))
    image_tensor, bev_tensor, true_speed = dataset[idx]
    
    with torch.no_grad():
        # Handle batch dimension
        pred_speed = model(image_tensor.unsqueeze(0).to(device), 
                           bev_tensor.unsqueeze(0).to(device))

    fig, ax = plt.subplots(1, 2, figsize=(16, 6))
    
    # 1. Image View (Handle both 1-cam and 6-cam)
    # If 4D (6, 3, 224, 224), take the first camera
    display_img = image_tensor[0] if image_tensor.dim() == 4 else image_tensor
    img_show = display_img.permute(1, 2, 0).cpu().numpy()
    img_show = (img_show * [0.229, 0.224, 0.225]) + [0.485, 0.456, 0.406]
    
    ax[0].imshow(np.clip(img_show, 0, 1))
    ax[0].set_title("On-Board Camera View", fontsize=14)
    ax[0].axis('off')
    
    # 2. LiDAR BEV
    ax[1].imshow(bev_tensor[1], cmap='magma')
    ax[1].set_title("LiDAR Bird's Eye View (Density)", fontsize=14)
    ax[1].axis('off')
    
    plt.suptitle(f"GROUND TRUTH: {true_speed.item():.2f} m/s  |  AI PREDICTION: {pred_speed.item():.2f} m/s", 
                 fontsize=18, fontweight='bold', y=1.05, color='#2c3e50')
    
    plt.tight_layout()
    plt.savefig('prediction_visual.png')
    plt.show()

# Run the dashboard
final_dashboard(model, dataset, device)

In [22]:
import cv2
from PIL import Image

def generate_surround_video(model, dataset, device, output_path='surround_prediction.mp4'):
    model.eval()
    # Define video properties: 1200x600 (3 images wide, 2 high)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, 10.0, (1200, 600))
    
    print("Stitching 6-Camera system into video...")

    # Process first 100 samples for the video
    for i in range(min(100, len(dataset))):
        cams, bev, speed = dataset[i]
        
        # Inference
        with torch.no_grad():
            pred = model(cams.unsqueeze(0).to(device), bev.unsqueeze(0).to(device)).item()
        
        # Process 6 images into a grid
        frames = []
        for j in range(6):
            # Convert tensor back to CV2 format (Un-normalize)
            img = cams[j].permute(1, 2, 0).cpu().numpy()
            img = (img * [0.229, 0.224, 0.225]) + [0.485, 0.456, 0.406]
            img = (np.clip(img, 0, 1) * 255).astype(np.uint8)
            img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
            img = cv2.resize(img, (400, 300))
            frames.append(img)
            
        # Stitch: Top (FL, F, FR) | Bottom (BL, B, BR)
        top = np.hstack(frames[:3])
        bottom = np.hstack(frames[3:])
        grid = np.vstack([top, bottom])
        
        # Add HUD Text
        color = (0, 255, 0) if abs(pred - speed.item()) < 1.0 else (0, 165, 255)
        cv2.putText(grid, f"AI PREDICTION: {pred:.2f} m/s", (400, 50), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)
        cv2.putText(grid, f"GROUND TRUTH: {speed.item():.2f} m/s", (400, 580), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
        
        out.write(grid)
        
    out.release()
    print(f"Video saved to {output_path}!")

# Run this to generate your 6D Video
generate_surround_video(model, dataset, device)

Stitching 6-Camera system into video...
Video saved to surround_prediction.mp4!


In [ ]:
import cv2
import matplotlib.cm as cm

def generate_K_presentation(model, dataset, device, output_path='K_presentation.mp4'):
    model.eval()
    # 3x3 Grid: 1200x900 (Each block is 400x300)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, 10.0, (1200, 900))
    
    # Mapping dataset indices to visual positions
    # Index in dataset image stack: 0:F, 1:FR, 2:BR, 3:B, 4:BL, 5:FL
    pos_map = {
        (0, 0): 5, # Top-Left: Front-Left
        (0, 1): 0, # Top-Mid: Front
        (0, 2): 1, # Top-Right: Front-Right
        (1, 0): 4, # Mid-Left: Back-Left
        (1, 2): 2, # Mid-Right: Back-Right
        (2, 1): 3  # Bot-Mid: Back
    }

    print("Rendering K Command Center Video...")
    for i in tqdm(range(min(150, len(dataset)))):
        cams, bev, speed = dataset[i]
        
        with torch.no_grad():
            pred = model(cams.unsqueeze(0).to(device), bev.unsqueeze(0).to(device)).item()

        # Create Canvas
        canvas = np.zeros((900, 1200, 3), dtype=np.uint8)
        
        # 1. Place 6 Cameras
        for (row, col), cam_idx in pos_map.items():
            img = cams[cam_idx].permute(1, 2, 0).cpu().numpy()
            img = (img * [0.229, 0.224, 0.225] + [0.485, 0.456, 0.406])
            img = (np.clip(img, 0, 1) * 255).astype(np.uint8)
            img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
            img = cv2.resize(img, (400, 300))
            canvas[row*300:(row+1)*300, col*400:(col+1)*400] = img

        # 2. Place LiDAR in Center (Row 1, Col 1)
        # Use a cool colormap (Magma) for the LiDAR density channel
        lidar_img = bev[1].cpu().numpy()
        lidar_color = (cm.magma(lidar_img)[:, :, :3] * 255).astype(np.uint8)
        lidar_color = cv2.cvtColor(lidar_color, cv2.COLOR_RGB2BGR)
        lidar_color = cv2.resize(lidar_color, (400, 300))
        
        # Add a "Tech" border to LiDAR
        cv2.rectangle(lidar_color, (0,0), (399,299), (0, 255, 255), 3)
        cv2.putText(lidar_color, "LIDAR BEV FUSION", (10, 280), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)
        canvas[300:600, 400:800] = lidar_color

        # 3. Add HUD Info (Bottom Left and Bottom Right)
        # Draw tech-y background boxes for text
        cv2.rectangle(canvas, (20, 800), (380, 880), (30, 30, 30), -1)
        cv2.rectangle(canvas, (820, 800), (1180, 880), (30, 30, 30), -1)
        
        # Speed Prediction
        color = (0, 255, 0) if abs(pred - speed.item()) < 0.8 else (0, 150, 255)
        cv2.putText(canvas, f"AI PREDICT: {pred:.2f} m/s", (40, 850), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)
        
        # Ground Truth
        cv2.putText(canvas, f"TRUE SPEED: {speed.item():.2f} m/s", (840, 850), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
        
        # Add a scanline effect or timestamp for "coolness"
        cv2.putText(canvas, f"FRAME: {i:03d} | SYSTEM: ACTIVE", (450, 50), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

        out.write(canvas)
        
    out.release()
    print(f"Presentation video saved to {output_path}")

# Run the final export
generate_K_presentation(model, dataset, device)

In [24]:
def plot_loss(loss_history):
    plt.figure(figsize=(10, 5))
    plt.plot(loss_history, label='Training Loss (Huber)')
    plt.xlabel('Iterations')
    plt.ylabel('Loss')
    plt.title('Fusion Model Convergence')
    plt.legend()
    plt.grid(True)
    plt.show()